In [1]:
import pandas as pd
import torch

from flags.lrh import FlagUnembeddingRepresentation
from flags.utils.memory import gc_cuda
from flags.utils.plotting import histplot_and_save

/home/pedro/Documents/flag-representation-hypothesis/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_IDS = {
    # model family | model ids
    # Llama 3.1
    "Llama 3.1 70B (Base x Instruct)": (
        "hugging-quants/Meta-Llama-3.1-70B-BNB-NF4-BF16",
        "hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
    ),
    "Llama 3.1 8B (Base x Instruct)": (
        "meta-llama/Meta-Llama-3.1-8B",
        "meta-llama/Meta-Llama-3.1-8B-Instruct",
    ),
    # Gemma 2
    "Gemma 2 27B (Base x Instruct)": ("google/gemma-2-27b", "google/gemma-2-27b-it"),
    "Gemma 2 9B (Base x Instruct)": ("google/gemma-2-9b", "google/gemma-2-9b-it"),
}

LEMMA_COUNT = 3  # {"gemma": 25, "llama": 55}
FLAG_RANK = 3

VALUE = "Similarity"
HUE = "Model Family"

In [3]:
def get_concepts(model_id):
    with gc_cuda():
        return FlagUnembeddingRepresentation.from_model_id(
            model_id, torch_dtype=torch.bfloat16
        ).get_all_concepts(min_lemmas_per_synset=LEMMA_COUNT, max_token_count=FLAG_RANK)


def get_similarities(id1, id2):
    return get_concepts(id1).similarity(get_concepts(id2)).float().cpu()


df = pd.concat(
    [
        pd.DataFrame({model_family: get_similarities(*model_ids)}).melt(
            var_name=HUE, value_name=VALUE
        )
        for model_family, model_ids in MODEL_IDS.items()
    ]
)

histplot_and_save("06_model_family_concept_space_comparison", df, VALUE, HUE)